In [7]:
pip install yfinance pandas matplotlib seaborn


In [8]:
pip install reportlab


In [9]:
# Importing required libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.pdfbase.pdfmetrics import stringWidth

# Importing Amazon stock data using yfinance
data = yf.download('AMZN', start='2020-01-01', end='2023-12-31')
data.index = pd.to_datetime(data.index)
data.ffill(inplace=True)

# Data Cleaning and Selecting Necessary Data
data['Normalized_Close'] = data['Close'] / data['Close'].iloc[0]
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['SMA_50'] = data['Close'].rolling(window=50).mean()
data['Daily Return'] = data['Close'].pct_change() * 100

# Visualization
sns.set(style="darkgrid")

def save_plot(filename, plot_func):
    plot_func()
    plt.tight_layout()
    plt.savefig(filename, bbox_inches='tight')
    plt.close()

# Closing Price
save_plot("closing_price.png", lambda: (
    plt.plot(data['Close'], label='AMZN Closing Price', color='teal'),
    plt.title('Amazon Stock Closing Price Over Time'),
    plt.xlabel('Date'), plt.ylabel('Price (USD)'), plt.legend()
))

# Price Distribution
save_plot("distribution_prices.png", lambda: (
    sns.histplot(data['Close'], kde=True, color='purple'),
    plt.title('Distribution of Amazon Stock Closing Prices')
))

# Correlation Heatmap
save_plot("correlation_heatmap.png", lambda: (
    sns.heatmap(data.corr(), annot=True, cmap='coolwarm'),
    plt.title('Correlation Between Variables')
))

# Moving Averages
save_plot("moving_averages.png", lambda: (
    plt.plot(data['Close'], label='Close Price', color='blue'),
    plt.plot(data['SMA_20'], label='20-Day SMA', color='orange'),
    plt.plot(data['SMA_50'], label='50-Day SMA', color='green'),
    plt.title('Amazon Stock with Moving Averages'), plt.legend()
))

# Daily Return Plot
save_plot("daily_return.png", lambda: (
    sns.lineplot(x=data.index, y=data['Daily Return'], color='darkred'),
    plt.title('Daily Return Over Time'), plt.xlabel('Date'), plt.ylabel('Daily Return (%)')
))

# Generating PDF Report
pdf_filename = "amazon_stock_analysis_report.pdf"
c = canvas.Canvas(pdf_filename, pagesize= letter )

width, height = letter
margin = 72
image_width = 5.5 * inch
image_height = 3 * inch
y_offset = height - margin

# Report Title and Introduction
title = "Amazon Stock Analysis Report (2020–2023)"
c.setFont("Helvetica-Bold", 20)
text_width = stringWidth(title, "Helvetica-Bold", 20)  # Match font size
x_centered = (width - text_width) / 2
c.drawString(x_centered, y_offset, title)
y_offset -= 30

intro = [
    "Amazon.com Inc. is a global leader in e-commerce and cloud services." ,
    "This report analyzes Amazon’s stock behavior from Jan 2020 to Dec 2023." ,
    "It includes visualizations of price trends, returns, and technical indicators.",
    ""
]
c.setFont("Helvetica", 14)
for line in intro:
    c.drawString(margin, y_offset, line)
    y_offset -= 16

# Function to add charts with titles and captions
def add_figure(fig_title, image_path, caption_lines):
    global y_offset
    if y_offset < image_height + 80:
        c.showPage()
        y_offset = height - margin

    c.setFont("Helvetica-Bold", 12)
    c.drawString(margin, y_offset, fig_title)
    y_offset -= 20

    c.drawImage(image_path, margin, y_offset - image_height, width=image_width, height=image_height)
    y_offset -= (image_height + 10)

    c.setFont("Helvetica", 10)
    for line in caption_lines:
        c.drawString(margin, y_offset, line)
        y_offset -= 20
    y_offset -= 20

# Insert all plots into the PDF
add_figure("Figure 1: Closing Price Over Time", "closing_price.png", [
    "Amazon’s stock price from 2020 to 2023 shows growth phases and market corrections.",
    "Key events include the pandemic boom and tech-sector fluctuations."
])

add_figure("Figure 2: Distribution of Closing Prices", "distribution_prices.png", [
    "Most closing prices lie within a central range with some high-price outliers.",
    "This distribution reflects Amazon’s relative price stability with periods of volatility."
])

add_figure("Figure 3: Correlation Between Variables", "correlation_heatmap.png", [
    "Strong positive correlation between SMAs and closing prices is observed.",
    "Daily returns are more independent, indicating higher volatility."
])

add_figure("Figure 4: Moving Averages", "moving_averages.png", [
    "Short-term (20-day) and long-term (50-day) SMAs reveal trend directions.",
    "Crossovers often signal shifts in momentum or trend reversals."
])

add_figure("Figure 5: Daily Return Over Time", "daily_return.png", [
    "Return spikes indicate impactful events like earnings or economic news.",
    "Volatility provides insight into risk and opportunity."
])

# Save the PDF
c.save()
print(f"PDF report generated: {pdf_filename}")


[*********************100%***********************]  1 of 1 completed


PDF report generated: amazon_stock_analysis_report.pdf
